In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, col, broadcast, avg, coalesce, lit, count
from pyspark.sql.types import StructType, StructField, StringType, ArrayType
from pyspark.sql import functions as F
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import time
import json

%run "/usr/local/spark/notebooks/00-spark-connection.ipynb"

# Optimize shuffle partitions based on your machine's cores
spark.conf.set("spark.sql.shuffle.partitions", "8")  # Adjust as needed

In [ ]:
start_time = time.time()

# Read enriched departures from Parquet file
df = spark.read.parquet("data/enriched_01.parquet")
print(f"Loaded Parquet data in {time.time() - start_time:.2f} seconds")

# Load line data
# Read the JSON file as a regular text file
start_time = time.time()
with open("data/relevantLines_with_stops.json", "r") as file:
    json_data = json.load(file)

# Create a list of rows
rows = [(line_id, line_info['name'], line_info['product']) for line_id, line_info in json_data.items()]

# Define the schema for the DataFrame
schema = StructType([
    StructField("line_id", StringType(), True),
    StructField("name", StringType(), True),
    StructField("product", StringType(), True)
])

# Create the DataFrame
lines_df = spark.createDataFrame(rows, schema)

# Show the first few rows to verify
lines_df.show(5)
print(f"Loaded line data in {time.time() - start_time:.2f} seconds")

# Join the enriched departures with the line information
start_time = time.time()
joined_df = df.join(lines_df, df.lineId == lines_df.line_id, "left") \
    .select(df["*"], 
            lines_df.name.alias("line_name"), 
            lines_df.product.alias("line_product"))
joined_df.show(5)
print(f"Joined data in {time.time() - start_time:.2f} seconds")

In [ ]:
# Calculate average delay grouped by product and collect example line names
delay_by_product = joined_df.groupBy("line_product") \
    .agg(F.avg("delay").alias("avg_delay"), 
         F.count("*").alias("count"),
         F.collect_set("line_name").alias("line_names")) \
    .orderBy(F.desc("avg_delay"))

# Function to limit array to 5 elements and format as string
def limit_and_format(arr):
    limited = arr[:5]
    return ", ".join(limited) + ("..." if len(arr) > 5 else "")

# Register the UDF
limit_and_format_udf = F.udf(limit_and_format, StringType())

# Apply the UDF to the line_names column
delay_by_product_with_examples = delay_by_product.withColumn(
    "example_lines", 
    limit_and_format_udf(F.col("line_names"))
)

# Show the results
delay_by_product_with_examples.select(
    "line_product", 
    F.round("avg_delay", 2).alias("avg_delay"), 
    "count", 
    "example_lines"
).show(truncate=False)

In [ ]:
# Merge regional and suburban products
merged_df = delay_by_product_with_examples.withColumn(
    "line_product",
    F.when(F.col("line_product").isin(["regional", "suburban"]), "suburban")
     .otherwise(F.col("line_product"))
)

# Recalculate the aggregations with the merged products
merged_delay_by_product = merged_df.groupBy("line_product") \
    .agg(F.avg("avg_delay").alias("avg_delay"), 
         F.sum("count").alias("count"),
         F.flatten(F.collect_list("line_names")).alias("line_names")) \
    .orderBy(F.desc("avg_delay"))

# Apply the UDF to the merged results
merged_delay_with_examples = merged_delay_by_product.withColumn(
    "example_lines", 
    limit_and_format_udf(F.col("line_names"))
)

# Show the results with merged regional and suburban
merged_delay_with_examples.select(
    "line_product", 
    F.round("avg_delay", 2).alias("avg_delay"), 
    "count", 
    "example_lines"
).show(truncate=False)


In [ ]:
# add count per day
days_diff = (df.agg(F.max("plannedWhen")).collect()[0][0] - df.agg(F.min("plannedWhen")).collect()[0][0]).days + 1

print(f"Days diff: {days_diff}")

merged_delay_with_examples = merged_delay_with_examples.withColumn(
    "count_per_day", 
    F.col("count") / days_diff
)

# Show the results with count per day
merged_delay_with_examples.select(
    "line_product", 
    F.round("avg_delay", 2).alias("avg_delay"), 
    "count", 
    F.round("count_per_day", 2).alias("count_per_day"), 
    "example_lines"
).show(truncate=False)

In [ ]:
# Convert Spark DataFrame to Pandas DataFrame
pdf = merged_delay_with_examples.select(
    "line_product", 
    F.round("avg_delay", 2).alias("avg_delay"), 
    F.round("count_per_day", 2).alias("count_per_day")
).toPandas()

# Plotting
fig, ax1 = plt.subplots(figsize=(14, 8))

# Add dashed y-grid
ax1.yaxis.grid(True, linestyle='--', alpha=0.7)

# Plot average delay on the primary y-axis
bar_width = 0.35
x = range(len(pdf['line_product']))
ax1.bar([i - bar_width/2 for i in x], pdf['avg_delay'], width=bar_width, color='salmon', label='Avg Delay')
ax1.set_xlabel('Line Product')
ax1.set_ylabel('Average Delay (seconds)', color='salmon')
ax1.tick_params(axis='y', labelcolor='salmon')
ax1.set_xticks(x)
ax1.set_xticklabels(pdf['line_product'])  # Use line products for x-axis labels

# Create a second y-axis for count per day
ax2 = ax1.twinx()
ax2.bar([i + bar_width/2 for i in x], pdf['count_per_day'], width=bar_width, color='skyblue', label='Usage per Day')
ax2.set_ylabel('Usage per Day', color='skyblue')
ax2.tick_params(axis='y', labelcolor='skyblue')

# Format y-axis labels to use commas as thousand separators
def format_func(value, tick_number):
    return f'{int(value):,}'

ax2.yaxis.set_major_formatter(FuncFormatter(format_func))

# Add legend
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

plt.title('Average Delay and Usage per Day by Line Product')

# Rotate and align the tick labels so they look better
plt.setp(ax1.get_xticklabels(), rotation=45, ha='right', rotation_mode='anchor')

# Adjust the subplot layout
fig.tight_layout()

# Show the plot
plt.show()